In [1]:
import pandas as pd
import numpy as np
import struct
import os
import matplotlib.pyplot as plt

In [2]:
# open file
fname = os.path.abspath(os.getcwd()) + "/STDF_data/main_Lot_1_Wafer_1_Oct_13_09h33m41s_STDF.stdf"

In [8]:
with open(fname,mode='rb') as file:
    row_data = file.read()
str_row_data=str(row_data)

# Filter Method

In [4]:
dict_pack = {
    'FAR':[0,10],
    'ATR':[0,20],
    'MIR':[1,10],
    'MRR':[1,20],
    'PCR':[1,30],
    'HBR':[1,40],
    'SBR':[1,50],
    'PMR':[1,60],
    'PGR':[1,62],
    'PLR':[1,63],
    'RDR':[1,70],
    'SDR':[1,80],
    'WIR':[2,10],
    'WRR':[2,20],
    'WCR':[2,30],
    'PIR':[5,10],
    'PRR':[5,20],
    'TSR':[10,30],
    'PTR':[15,10],
    'MPR':[15,15],
    'FTR':[15,20],
    'BPS':[20,10],
    'EPS':[20,20],
    'GDR':[50,10],
    'DTR':[50,30]
}
#dict_pack['PRR'][0]

In [6]:
def struct_packing(rtype,subtype):
    packing = str(struct.pack("BB",rtype,subtype))
    return packing[2:len(packing)-1]

def data_mark(string_row_data,key_header,key):
    key_list = string_row_data.split(key_header)    #will cut key_header

    for n in range(0,len(dict_position[key])):
        if dict_position[key][n][1]==1:            
            key_list[n] = key_list[n][:len(key_list[n])-8] + "|" + key_list[n][len(key_list[n])-8:]
            key_list[n] = key_list[n]+"%"+key+":"+key_header
        else:
            key_list[n] = key_list[n]+key_header
    
    return "%".join(key_list)


def debug_by_positionrule_2():
    #/need execute before data_marking/
    #position locate
    dict_position = {}

    for key in dict_pack.keys():
        header = struct_packing(dict_pack[key][0],dict_pack[key][1])
        group_list = str_row_data.split(header)
        group_count = 0
        dict_position[key]=[]
        for num in range(0, len(group_list)-1):
            group_count = group_count + len(group_list[num])
            add_group = [group_count,1]
            dict_position[key].append(add_group)
    
    #ATR:bewteen FAR,MIR
    if dict_position['ATR'][0][0] < dict_position['ATR'][1][0]:
        dict_position['ATR'][1][1]=0
    #MRR:need last
    elif dict_position['MRR'][0][0] < dict_position['MRR'][1][0]:
        dict_position['MRR'][0][1]=0
    #SBR:after HBR
    for l in dict_position['SBR']:
        if l[0] < dict_position['HBR'][0][0]:
            l[1]=0
    #PIR:after PRR
    for l in dict_position['PIR']:
        if l[0] < dict_position['PRR'][0][0]:
            l[1]=0
    
    return dict_position


def extract_data(processed_data):
    record_type_split = processed_data.split("|")
    for data in range(1,len(record_type_split)):
        data_extracted = record_type_split[data].split("%")
    return data_extracted



# execute

In [9]:
dict_position = debug_by_positionrule_2()

In [ ]:
# read binary data
with open(fname,mode='rb') as file:
    row_data = file.read()
str_row_data=str(row_data)

dict_position = debug_by_positionrule_2()

# data mark
processed_data = str_row_data
for num_key in range(0,25):
    key = list(dict_pack.keys())[num_key]
    header = struct_packing(dict_pack[key][0],dict_pack[key][1])
    processed_data = data_mark(processed_data,header,key)
processed_data

In [ ]:
#check data integrity
record_type_split = processed_data.split("|")
for data in range(1,len(record_type_split)):
    data_extracted = record_type_split[data].split("%")
    print(len(data_extracted),data_extracted)

In [ ]:
head_part = []
record_type = []
body_part = []
missing_data = []

record_type_split = processed_data.split("|")
for data in range(1,len(record_type_split)):
    data_extracted = record_type_split[data].split("%")
        
    if len(data_extracted) == 3:
        head_part.append(data_extracted[0])
        record_type.append(data_extracted[1].split(":")[0])
        body_part.append(data_extracted[2])
    elif len(data_extracted) == 4:
        head_part.append(data_extracted[0])
        record_type.append(data_extracted[1].split(":")[0])
        body_part.append(data_extracted[2]+data_extracted[3])
    else:
        missing_data.append(data_extracted)       
        
print("h",head_part)
print("\nr",record_type)
print("\nb",body_part)
print("\nm",missing_data)

print(len(head_part))
print(len(record_type))
print(len(body_part))

In [ ]:
zip_data = zip(head_part,record_type,body_part)
zip_data
#list_data = list(zip_data)

In [ ]:
df = pd.DataFrame(columns=['head_part','record_type','body_part'])
df['head_part'] = head_part
df['record_type'] = record_type
df['body_part'] = body_part
df

### str 轉回 binary 出現問題
剪貼簿暫解

In [ ]:
filter = df['record_type']=='PRR'
n=0
for line in df[filter].body_part:
    n += 1
    print(line)
print(n)

In [ ]:
p1="\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfb\xff\xfc\xff\x03\x02\x00\x00\x011\x050.000\x00"
p2="\x01\x01\x00\x00\x00\x01\x00\x01\x00\xff\xff\xfc\xff\x01\x00\x00\x00\x012\x051.750\x00"
p3="\x01\x01\x00\x00\x00\x01\x00\x01\x00\x04\x00\xfc\xff\x01\x00\x00\x00\x013\x052.201\x00"
p4="\x01\x01\x00\x00\x00\x01\x00\x01\x00\x06\x00\xfc\xff\x01\x00\x00\x00\x014\x052.803\x00"
p5="\x01\x01\x00\x00\x00\x01\x00\x01\x00\x07\x00\xfc\xff\x01\x00\x00\x00\x015\x052.547\x00"
p6="\x01\x01\x00\x00\x00\x01\x00\x01\x00\xf9\xff\xfd\xff\x01\x00\x00\x00\x016\x052.702\x00"
p7="\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfd\xff\xfd\xff\x01\x00\x00\x00\x017\x052.700\x00"
p8="\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfe\xff\xfd\xff\x01\x00\x00\x00\x018\x052.101\x00"
p9="\x01\x01\x00\x00\x00\x01\x00\x01\x00\x03\x00\xfd\xff\x01\x00\x00\x00\x019\x052.600\x00"
p10="\x01\x01\x00\x00\x00\x01\x00\x01\x00\x04\x00\xfd\xff\x01\x00\x00\x00\x0210\x052.765\x00"
p11="\x01\x01\x00\x00\x00\x01\x00\x01\x00\x06\x00\xfd\xff\x01\x00\x00\x00\x0211\x052.704\x00"
p12="\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfa\xff\x00\x00\x01\x00\x00\x00\x0212\x052.802\x00"
p13="\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfc\xff\x00\x00\x01\x00\x00\x00\x0213\x052.304\x00"

p_list = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13]
prr = ['U1','U1','B1','U2','U2','U2','I2','I2','U4','Cn','Cn','Bn']
prr_head = ['head_num','site_num','part_flg','num_test','hard_bin','soft_bin','x_coord','y_coord','test_t','part_id','part_txt','part_fix']
fmt = "<BBBHHHhhL"

head_num = []
site_num = []
part_flg = []
num_test = []
hard_bin = []
soft_bin = []
x_coord = []
y_coord = []
test_t = []
part_id = []
part_txt = []
part_fix = []

df = pd.DataFrame(columns=prr_head)

for n in range(0,len(p_list)):
    data = p_list[n].encode("latin1")
    dec = struct.unpack(fmt,data[:17])
    dec_data = [*dec,]

    dec_data.append(data[18:18+len(str(n+1))].decode())
    
    f_txt=data[19+len(str(n+1)):24+len(str(n+1))].decode()
    dec_data.append(float(f_txt))

    dec_data.append([])

    head_num.append(dec_data[0])
    site_num.append(dec_data[1])
    part_flg.append(dec_data[2])
    num_test.append(dec_data[3])
    hard_bin.append(dec_data[4])
    soft_bin.append(dec_data[5])
    x_coord.append(dec_data[6])
    y_coord.append(dec_data[7])
    test_t.append(dec_data[8])
    part_id.append(dec_data[9])
    part_txt.append(dec_data[10])
    part_fix.append(dec_data[11])


df['head_num']=head_num
df['site_num']=site_num
df['part_flg']=part_flg
df['num_test']=num_test
df['hard_bin']=hard_bin
df['soft_bin']=soft_bin
df['x_coord']=x_coord
df['y_coord']=y_coord
df['test_t']=test_t
df['part_id']=part_id
df['part_txt']=part_txt
df['part_fix']=part_fix

df

In [ ]:
## Visualization

#data list
x=df["x_coord"]
y=df["y_coord"]
id=df["part_id"]

# plt setting
fig = plt.figure() 
ax = fig.add_subplot(1, 1, 1) 

plt.title("Distribution of Points")
plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")

#scatter plot
ax.scatter(x, y, color="blue", s=50, marker="o", alpha=0.2)

#show id on grapg
for i in range(len(x)):
    plt.annotate(id[i], xy=(x[i], y[i]), color="black", size=12, ha="center", va="center")

#add grid
ax.set_yticks(np.arange(-8,8,1))
ax.set_yticks(np.arange(-8,8,1)-0.5, minor=True)

ax.set_xticks(np.arange(-8,8,1))
ax.set_xticks(np.arange(-8,8,1)+0.5, minor=True)
ax.grid(True, which="minor")
ax.set_aspect("equal")

plt.show()